In [15]:
import networkx as nx
import heapq
from collections import defaultdict, deque

class ChristofidesSolver: 
    def __init__(self, filename):

        distance_matrix = []

        with open(filename, 'r') as f:
            lines = f.readlines()
            read_distances = False  

            for line in lines: 
                line = line.strip()
                line = line.replace(':', ' ')

                if line.startswith('EOF'):
                    break
                elif line.startswith('EDGE_WEIGHT_SECTION'):
                    read_distances = True
                elif read_distances:
                    elements = line.split()
                    distance_matrix.append([int(e) for e in elements])

        self.graph = distance_matrix
        self.n = len(distance_matrix)

    def prim_mst(self):

        self.mst_edges = []  # Stores (u, v, weight) edges in MST
        total_weight = 0
        visited = [False] * self.n
        min_heap = [(0, 0, -1)]  # (weight, vertex, parent)

        while len(self.mst_edges) < self.n - 1:
            weight, u, parent = heapq.heappop(min_heap)

            if visited[u]:
                continue
            
            visited[u] = True
            if parent != -1:
                self.mst_edges.append((parent, u, weight))
                total_weight += weight

            for v in range(self.n):
                if not visited[v] and self.graph[u][v] > 0:
                    heapq.heappush(min_heap, (self.graph[u][v], v, u))

    def find_odd_degree_vertices(self):
        degree = [0] * self.n

        for u, v, _ in self.mst_edges:
            degree[u] += 1
            degree[v] += 1

        self.odd_vertices = []
        for v in range(self.n):
            if degree[v] % 2 == 1:
                self.odd_vertices.append(v) 

    def minimum_weight_perfect_matching(self):
        """Find the Minimum Weight Perfect Matching (MWPM) for odd-degree vertices."""
        G = nx.Graph()
        for i in range(len(self.odd_vertices)):
            for j in range(i + 1, len(self.odd_vertices)):
                u, v = self.odd_vertices[i], self.odd_vertices[j]
                weight = self.graph[u][v]
                G.add_edge(u, v, weight=weight)
        mwpm = nx.algorithms.matching.min_weight_matching(G, maxcardinality=True, weight="weight")

        self.matching_edges = []
        for u, v in mwpm:
            self.matching_edges.append((u, v, self.graph[u][v]))

    def find_euler_circuit(self):

        self.mst_mwpm_graph = self.mst_edges.copy()  # Make a copy of mst_graph
        self.mst_mwpm_graph.extend(self.matching_edges)  # Append matching_edges to mst_graph

        # Initialize the graph with the adjacency list
        adj = defaultdict(deque)
    
        # Building the graph
        for u, v, _ in self.mst_mwpm_graph:
            adj[u].append(v)
            adj[v].append(u)

        # Check if all vertices have an even degree
        for node in adj:
            if len(adj[node]) % 2 != 0:
                print(f"Vertex {node} has an odd degree, so no Euler circuit exists.")
                return None

        # Non-recursive Hierholzer's algorithm to find Eulerian circuit
        self.euler_circuit = []
        # Start from any vertex that has edges (pick the first one)
        start_node = next(iter(adj))
        current_path = deque([start_node])
    
        while current_path:
            u = current_path[-1]

            # If there are still neighbors (edges to follow)
            if adj[u]:
                # Get the next vertex to visit
                v = adj[u].popleft()  # Get a neighbor (remove the edge)
                adj[v].remove(u)  # Remove the reverse edge
                current_path.append(v)  # Move to the next vertex
            else:
                # If there are no more neighbors, add the vertex to the circuit
                self.euler_circuit.append(current_path.pop())
    
        # Since we processed the circuit in reverse order, we reverse it to get the correct order
        self.euler_circuit.reverse()

    def christofides(self):

        self.prim_mst()

        self.find_odd_degree_vertices()

        self.minimum_weight_perfect_matching()

        self.find_euler_circuit()

        unique_tour = []
        cost = 0
        seen = set()
 
        for i, vertex in enumerate(self.euler_circuit):
            if vertex not in seen:
                unique_tour.append(vertex)
                seen.add(vertex)

                # If not the first vertex, calculate the cost from the last added vertex
                if len(unique_tour) > 1:
                    prev_vertex = unique_tour[-2]
                    cost += self.graph[prev_vertex][vertex]  # Assuming graph is adjacency matrix

        # To return to the start vertex (completing the cycle)
        cost += self.graph[unique_tour[-1]][unique_tour[0]]

        return cost, unique_tour

In [16]:
filename = "../tsplib_converted_instances/01_small/gr17_converted.tsp"

# Solve using 2-Opt 
christofides_solver = ChristofidesSolver(filename)
ch_cost, ch_path = christofides_solver.christofides()

print("\nChristofides-Solver - Minimum Cost:", ch_path)
print("\nPath: ", ch_cost)


Christofides-Solver - Minimum Cost: [0, 12, 6, 7, 5, 10, 4, 1, 9, 2, 14, 13, 16, 3, 8, 11, 15]

Path:  2190
